In [92]:
# automatically load changes from the sandbox_steps module
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
import sys
import os
from e2b_desktop import Sandbox, CommandExitException
from dotenv import load_dotenv
load_dotenv()

sys.path.append(os.path.abspath('..')) # Add the parent directory (project root) to sys.path

from counter_strike.install_cs import install_cs_1_6, connect_to_server, choose_team
from counter_strike.image_handling import get_screenshot_message, draw_point, get_screenshot

from llms.models import AimingModel

E2B_API_KEY = os.environ.get("E2B_API_KEY")
CS_SERVER_IP = os.environ.get("CS_SERVER_IP")


In [130]:
# With custom configuration
desktop = Sandbox(
    display=":0",  # Custom display (defaults to :0)
    resolution=(1920, 1080),  # Custom resolution
    timeout = 3600) 

desktop.stream.start()

# Get stream URL
url = desktop.stream.get_url()
print(url)

# only viewing 
url_view = desktop.stream.get_url(view_only=True)
print(url_view)

https://6080-ikxn0iut8aa7xadj6q9yp-5377ba0b.e2b.app/vnc.html?autoconnect=true&resize=scale
https://6080-ikxn0iut8aa7xadj6q9yp-5377ba0b.e2b.app/vnc.html?autoconnect=true&view_only=true&resize=scale


In [131]:
install_cs_1_6(desktop=desktop)
connect_to_server(desktop=desktop, ip_address=CS_SERVER_IP)
choose_team(desktop=desktop)

In [132]:
connect_to_server(desktop=desktop, ip_address=CS_SERVER_IP)
choose_team(desktop=desktop)

Waiting for 150 secs for map download...


In [65]:
#desktop.write(text="ddddddddddddddddd", delay_in_ms=0, chunk_size=50)
desktop.write(text="dddddddddddddddddddd")
desktop.write(text="aaaaaaaaaaaaaaaaaaaa")
desktop.write(text="wwwwwwwwwwwwwwwwwwww")

In [177]:
screenshot_message = get_screenshot_message(desktop, filename=SCREENSHOT_PATH)

In [178]:
aiming_model.complete(screenshot_message)

Model: qwen/qwen2.5-vl-32b-instruct
Provider: Fireworks


('```json\nnull\n```',
 ChatCompletion(id='gen-1746900812-cu8qrFuRA8RpmWSepJL2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\nnull\n```', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1746900812, model='qwen/qwen2.5-vl-32b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=7, prompt_tokens=2790, total_tokens=2797, completion_tokens_details=None, prompt_tokens_details=None), provider='Fireworks'))

In [179]:
output = process_models_concurrently(screenshot_message, aiming_model, gameplay_model)
print(output)

Model: qwen/qwen2.5-vl-32b-instruct
Provider: Fireworks
(None, [ChatCompletionMessageToolCall(id='tool_0_move_tool', function=Function(arguments='{"key_sequence":"wwass"}', name='move_tool'), type='function', index=0)], 1.1472323999914806, 1.4249587999947835)


In [3]:
desktop.kill()

In [150]:
from counter_strike.image_handling import get_mouse_movements
from counter_strike.controls import aim, shoot

from llms.models import AimingModel

aiming_model = AimingModel()

In [151]:
from llms.tools import MoveTool
from llms.models import OpenRouterGameplayModel

move_tool = MoveTool(desktop=desktop)
tools_move = {move_tool.name: move_tool}

gameplay_model = OpenRouterGameplayModel(tools=tools_move, model="google/gemini-2.5-flash-preview")
move_tool.name

'move_tool'

In [121]:
for i in range(10):
    print(f"Iteration {i}")
    screenshot_message = get_screenshot_message(desktop, filename="../images/screenshot.jpg")

    point_json, _ = aiming_model.complete(user_messages=screenshot_message, debug=True)
    print(point_json)
    coords = aiming_model.parse_point_json(point_json)


    if not coords:
        content, response, tool_calls = gameplay_model.complete(user_messages=screenshot_message) # will use tools to move around
        if tool_calls:
            gameplay_model._handle_tool_calls(tool_calls=tool_calls)
        else:
            print(f"The gameplay model did not return any tool calls. Response: {response}")

    else:
        draw_point(point = coords, image_path="../images/screenshot.jpg", output_path="../images/screenshot_annotated.jpg")

        mouse_movements = get_mouse_movements(coords=coords)
        aim(mouse_movements, desktop=desktop)
        shoot(desktop=desktop)

Iteration 0
ChatCompletion(id='gen-1746896217-GaDhiLLObAzcK90Rpo6R', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='None', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1746896217, model='qwen/qwen2.5-vl-72b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=2, prompt_tokens=2790, total_tokens=2792, completion_tokens_details=None, prompt_tokens_details=None), provider='Parasail')
None
[{'role': 'user', 'content': 'Current map: aim_map_2010.Design five consecutive movement steps to locate the enemy: 1) Choose a combined sequence of 5 keys (w/a/s/d) per move. 2) Return your answer by calling move_tool with the \'key_sequence\' parameter. Example: {"name": "move_tool", "arguments": {"key_sequence": "wwaad"}}'}]
Arguments: {'key_sequence': 'wwwww'}
moving with 

KeyboardInterrupt: 

In [ ]:
from datetime import datetime

class ImageLoggingSettings:
    def __init__(self, base_path="../images"):
        """
        Initializes settings for image logging.
        Creates a unique directory for this session based on the current date and time.
        """
        self.session_timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.session_log_dir = os.path.join(base_path, self.session_timestamp_str)
        os.makedirs(self.session_log_dir, exist_ok=True)
        
        # These will store the paths for the current iteration's images
        self._current_screenshot_path = None
        self._current_annotated_screenshot_path = None
        
        print(f"Image logging session directory initialized: {self.session_log_dir}")

    def generate_new_paths_for_iteration(self):
        """
        Generates and stores new, unique paths for a screenshot and its annotation
        for the current processing iteration. This method should be called once at the
        start of each iteration that requires image logging.
        """
        iteration_file_timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        screenshot_filename = f"screenshot_{iteration_file_timestamp_str}.jpg"
        self._current_screenshot_path = os.path.join(self.session_log_dir, screenshot_filename)
        
        # Ensure annotated version uses the same unique timestamp as the screenshot
        annotated_filename = f"screenshot_annotated_{iteration_file_timestamp_str}.jpg"
        self._current_annotated_screenshot_path = os.path.join(self.session_log_dir, annotated_filename)

    def get_screenshot_path(self) -> str:
        """
        Returns the full path for the screenshot of the current iteration.
        Ensure generate_new_paths_for_iteration() has been called for the current iteration.
        """
        if self._current_screenshot_path is None:
            # This case should ideally not be hit if generate_new_paths_for_iteration is called correctly.
            # Consider raising an error or logging a more severe warning if strict ordering is critical.
            print("Warning: get_screenshot_path() accessed before paths were generated for this iteration. Generating now.")
            self.generate_new_paths_for_iteration()
        return self._current_screenshot_path

    def get_annotated_screenshot_path(self) -> str:
        """
        Returns the full path for the annotated screenshot of the current iteration.
        Ensure generate_new_paths_for_iteration() has been called for the current iteration.
        """
        if self._current_annotated_screenshot_path is None:
            print("Warning: get_annotated_screenshot_path() accessed before paths were generated for this iteration. Generating now.")
            self.generate_new_paths_for_iteration()
        return self._current_annotated_screenshot_path

In [ ]:
import concurrent.futures
import time 

def process_models_concurrently(screenshot_message, aiming_model, gameplay_model):
    """
    Runs aiming and gameplay models concurrently, prioritizing aiming results.
    Returns coordinates if found, otherwise tool_calls from gameplay.
    """
    tool_calls_output = None
    aiming_model_time = 0
    gameplay_model_time = 0 # Will only be set if gameplay_model.result() is called and used

    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        time_aiming_start = time.perf_counter()
        future_aiming = executor.submit(aiming_model.complete, user_messages=screenshot_message)

        time_gameplay_submit = time.perf_counter()
        future_gameplay = executor.submit(gameplay_model.complete, user_messages=screenshot_message)

        # Wait for aiming model to complete and get its execution time
        point_json, _ = future_aiming.result()
        time_aiming_end = time.perf_counter()
        aiming_model_time = time_aiming_end - time_aiming_start
        
        parsed_coords = aiming_model.parse_point_json(point_json)

        if parsed_coords:
            # If coords are found, we don't necessarily wait for gameplay.
            if not future_gameplay.done():
                future_gameplay.cancel()
            else:
                try:
                    _, _, _ = future_gameplay.result(timeout=0.01) # Quick check
                except (concurrent.futures.TimeoutError, concurrent.futures.CancelledError):
                    pass 
        else:
                _, _, tool_calls_output = future_gameplay.result()
                time_gameplay_wait_end = time.perf_counter()
                # this time can be wrong
                gameplay_model_time = time_gameplay_wait_end - time_gameplay_submit

    return parsed_coords, tool_calls_output, aiming_model_time, gameplay_model_time

# --- Configuration example #
SCREENSHOT_PATH = "../images/{log_dir_date_time}/screenshot_{datetime}.jpg"
ANNOTATED_SCREENSHOT_PATH = "../images/{log_dir_date_time}/screenshot_annotated_{datime}.jpg"

def perform_aiming_sequence(coords, desktop):
    """
    Executes the sequence of actions when coordinates are available.
    """
    # print(f"Coordinates found: {coords}. Proceeding with aiming and shooting.") # Less verbose
    draw_point(point=coords, image_path=SCREENSHOT_PATH, output_path=ANNOTATED_SCREENSHOT_PATH)
    mouse_movements = get_mouse_movements(coords=coords)
    aim(mouse_movements, desktop=desktop)
    shoot(desktop=desktop)

def handle_gameplay_actions(tool_calls, gameplay_model_instance): # Added gameplay_model_instance
    """
    Handles the actions based on tool_calls from the gameplay model.
    """
    if tool_calls:
        # print(f"No coordinates. Using tool_calls: {tool_calls}") # Less verbose
        gameplay_model_instance._handle_tool_calls(tool_calls=tool_calls)

    else:
        print("No coordinates found and no tool calls to execute from gameplay model.")

def run_game_loop(aiming_model, gameplay_model, desktop, iterations=10):
    """
    Main loop for the game interaction with performance logging.
    """
    total_loop_time_all_iterations = 0

    for i in range(iterations):
        iteration_start_time = time.perf_counter()
        print(f"\n--- Iteration {i + 1} ---")

        # 1. Screenshot timing
        screenshot_start_time = time.perf_counter()
        screenshot_message = get_screenshot_message(desktop, filename=SCREENSHOT_PATH) # HEEEEEEEEEEEEEEERE
        screenshot_end_time = time.perf_counter()
        screenshot_time = screenshot_end_time - screenshot_start_time
        print(f"  [Time] Screenshot: {screenshot_time:.4f}s")

        # 2. Process models concurrently and get their timings
        coords, tool_calls, aiming_time, gameplay_time = process_models_concurrently(
            screenshot_message,
            aiming_model,
            gameplay_model
        )
        print(f"  [Time] Aiming Model: {aiming_time:.4f}s")

        action_taken = "None"
        if coords:
            print(f"  [Action] Coords found: {coords}. Aiming & Shooting.")
            perform_aiming_sequence(coords, desktop) # HEEEEEEEEEERE
            action_taken = "Aim & Shoot"
        else: # No coords, check tool_calls
            if tool_calls:
                print(f"  [Action] No Coords. Using Gameplay Model Tool Calls.")
                if gameplay_time > 0: # Log gameplay time only if its result was used
                    print(f"  [Time] Gameplay Model: {gameplay_time:.4f}s")
                handle_gameplay_actions(tool_calls, gameplay_model) # Pass gameplay_model instance
                action_taken = f"Tool Calls: {tool_calls}"
            else:
                print(f"  [Action] No Coords, No Tool Calls.")
                if gameplay_time > 0: # Still log if it ran and we waited for it
                     print(f"  [Time] Gameplay Model (no valid output): {gameplay_time:.4f}s")
                action_taken = "No Action"

        iteration_end_time = time.perf_counter()
        iteration_total_time = iteration_end_time - iteration_start_time
        total_loop_time_all_iterations += iteration_total_time
        print(f" Action taken: {action_taken}")
        print(f"  [Time] Iteration {i+1} Total: {iteration_total_time:.4f}s")

In [180]:
run_game_loop(
    aiming_model=aiming_model,
    gameplay_model=gameplay_model,
    desktop=desktop,
    iterations=20 # For demonstration
) 


--- Iteration 1 ---
  [Time] Screenshot: 0.9344s
Model: qwen/qwen2.5-vl-32b-instruct
Provider: Fireworks
  [Time] Aiming Model: 1.5408s
  [Action] No Coords. Using Gameplay Model Tool Calls.
  [Time] Gameplay Model: 1.5384s
 Action taken: Tool Calls: [ChatCompletionMessageToolCall(id='tool_0_move_tool', function=Function(arguments='{"key_sequence":"wwwww"}', name='move_tool'), type='function', index=0)]
  [Time] Iteration 1 Total: 4.8935s

--- Iteration 2 ---
  [Time] Screenshot: 1.5854s
Model: qwen/qwen2.5-vl-32b-instruct
Provider: Fireworks
  [Time] Aiming Model: 1.8823s
  [Action] No Coords. Using Gameplay Model Tool Calls.
  [Time] Gameplay Model: 2.3514s
 Action taken: Tool Calls: [ChatCompletionMessageToolCall(id='tool_0_move_tool', function=Function(arguments='{"key_sequence":"wwddd"}', name='move_tool'), type='function', index=0)]
  [Time] Iteration 2 Total: 6.4349s

--- Iteration 3 ---
  [Time] Screenshot: 1.9548s
Model: qwen/qwen2.5-vl-32b-instruct
Provider: Fireworks
  [Tim

KeyboardInterrupt: 

In [ ]:
import concurrent.futures

for i in range(10):
    print(f"Iteration {i+1}") # Changed None to i+1 for clarity
    screenshot_message = get_screenshot_message(desktop, filename="../images/screenshot.jpg")

    coords = None
    point_json = None
    content = None
    tool_calls = None
    aiming_done_first_and_valid = False

    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        future_aiming = executor.submit(aiming_model.complete, user_messages=screenshot_message, debug=True)
        future_gameplay = executor.submit(gameplay_model.complete, user_messages=screenshot_message)

        futures = [future_aiming, future_gameplay]
        future_to_task = {future_aiming: "aiming", future_gameplay: "gameplay"}

        try:
            point_json_result, _ = future_aiming.result(timeout=None) 
            print(f"Aiming model output: {point_json_result}")
            parsed_coords = aiming_model.parse_point_json(point_json_result)
            if parsed_coords:
                coords = parsed_coords
                aiming_done_first_and_valid = True
                # If coords are found, we can cancel the gameplay future if it's not needed
                # and not already running or completed.
                if not future_gameplay.done():
                    future_gameplay.cancel()
                else: # if it's already done, get its result to avoid unretrieved exception
                    try:
                        content_result, response, tool_calls_result = future_gameplay.result(timeout=0.1)
                        content = content_result
                        tool_calls = tool_calls_result
                        print("Gameplay model completed even though coords were found.")
                    except (concurrent.futures.TimeoutError, concurrent.futures.CancelledError):
                        print("Gameplay model was cancelled or timed out after aiming was successful.")
                    except Exception as e:
                        print(f"Error retrieving result from already done gameplay future: {e}")


        except Exception as e:
            print(f"Error in aiming_model.complete: {e}")

        # If coords were found and processed, we skip waiting for gameplay if it was cancelled
        if aiming_done_first_and_valid:
            pass # Handled above
        else:
            # If coords were not found from aiming_model (either it failed or returned no coords)
            # we now need the result from gameplay_model
            print("No valid coords from aiming model, waiting for gameplay model.")
            try:
                # Wait for the gameplay future to complete if it hasn't been processed
                if not future_gameplay.done() or not tool_calls: # check if tool_calls already populated
                    content_result, _, tool_calls_result = future_gameplay.result(timeout=None) # Wait indefinitely
                    content = content_result
                    tool_calls = tool_calls_result
                    print(f"Gameplay model output: content='{content}', tool_calls='{tool_calls}'")
            except concurrent.futures.CancelledError:
                print("Gameplay model task was cancelled.")
            except Exception as e:
                print(f"Error in gameplay_model.complete: {e}")


    # Logic based on the results
    if coords:
        print(f"Coordinates found: {coords}. Proceeding with aiming and shooting.")
        draw_point(point=coords, image_path="../images/screenshot.jpg", output_path="../images/screenshot_annotated.jpg")
        mouse_movements = get_mouse_movements(coords=coords)
        aim(mouse_movements, desktop=desktop)
        shoot(desktop=desktop)
    elif tool_calls: # If no coords, but we have tool_calls from the gameplay model
        print(f"No coordinates. Using tool_calls: {tool_calls}")
        gameplay_model._handle_tool_calls(tool_calls=tool_calls)
    else:
        print(f"The gameplay model did not return any tool calls. Response.") # print the response here.

    print("-" * 30) # Separator for iterations

Iteration 1
[{'role': 'user', 'content': 'Current map: aim_map_2010.Design five consecutive movement steps to locate the enemy: 1) Choose a combined sequence of 5 keys (w/a/s/d) per move. 2) Return your answer by calling move_tool with the \'key_sequence\' parameter. Example: {"name": "move_tool", "arguments": {"key_sequence": "wwaad"}}'}]
ChatCompletion(id='gen-1746896490-iwwebrGcnOAtNf09cx0H', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{"point": {"x": "1065", "y": "542"}}\n```', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1746896490, model='qwen/qwen2.5-vl-72b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=25, prompt_tokens=2790, total_tokens=2815, completion_tokens_details=None, prompt_tokens_details=None), provider='Parasail')
Aiming 

KeyboardInterrupt: 